# Peer-graded Assignment: Build a Regression Model in Keras
## Activities:
Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error  as the loss function.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the
train_test_split
helper function from Scikit-learn.

2. Train the model on the training data using 50 epochs.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

5. Report the mean and the standard deviation of the mean squared errors.


### 1. Import Libraries

In [74]:
# Import libs
import numpy as np
import pandas as pd

### 2. Load the Dataset

In [75]:
# Load the dataset in a pandas dataframe
data_file_path = 'concrete_data.csv'
data = pd.read_csv(data_file_path)

In [76]:
data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


### 3. Data Overview and Summary Statistics

In [77]:
# Check basic information and first few rows
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Cement              1030 non-null   float64
 1   Blast Furnace Slag  1030 non-null   float64
 2   Fly Ash             1030 non-null   float64
 3   Water               1030 non-null   float64
 4   Superplasticizer    1030 non-null   float64
 5   Coarse Aggregate    1030 non-null   float64
 6   Fine Aggregate      1030 non-null   float64
 7   Age                 1030 non-null   int64  
 8   Strength            1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.6 KB


In [78]:
# Summary statistics to check basic stats
data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [79]:
# To check for any missing data within the records
data.isnull().sum()

,0
Cement,0
Blast Furnace Slag,0
Fly Ash,0
Water,0
Superplasticizer,0
Coarse Aggregate,0
Fine Aggregate,0
Age,0
Strength,0


The data looks very clean and is ready to be used to build our model.

### 4. Split the dataset into Predictors and Target
The target variable in this problem is the concrete sample strength. Therefore, our predictors will be rest of the columns within our dataset.



In [80]:
data_columns = data.columns
predictors = data[data_columns[data_columns !=  'Strength']] # all columns except the Strength
target = data['Strength'] # target Strength column

print("The shape of predictors dataframe:", predictors.shape)

The shape of predictors dataframe: (1030, 8)


In [81]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [82]:
target.head()

,Strength
0,79.99
1,61.89
2,40.27
3,41.05
4,44.30


In [83]:
# Number input columns to the model predictors
n_cols = predictors.shape[1]
n_cols

8

Normalize the data by substracting the mean and dividing by the standard deviation

In [93]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


### 5. Build Regression Model Using Keras
- Import Keras and packages to build linear model
- One hidden layer of 10 nodes, and a ReLU activation function
- Use the adam optimizer and the mean squared error as the loss function.

In [94]:
# Import keras and essentail packages for developing the linear model
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

In [95]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))

    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

### 6. Split the Train and Test Dataset
- Randomly split the data into a training and test sets by holding 30% of the data for testing.

In [97]:
# import scikit-learn in order to randomly split the data into a training and test sets
from sklearn.model_selection import train_test_split

# split the training and test data
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

### 7. Train and Test the Network
- Train the model on the training data using 50 epochs

In [98]:
# build the model
model = regression_model()

In [99]:
# fit the model
model.fit(X_train, y_train, epochs=50, verbose=1)

Epoch 1/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1582.1195
Epoch 2/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1475.3872
Epoch 3/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1493.5037
Epoch 4/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1579.7690
Epoch 5/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1573.6239
Epoch 6/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1530.9893
Epoch 7/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1494.9464
Epoch 8/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1436.4583
Epoch 9/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1423.6418 
Epoch 10/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1435.5149 
Epoch 11/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1417.4298 
Epoch 12/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1357.3611 
Epoch 13/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1379.9019 
Epoch 14/50
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1388.0151
Epoch 15/50
23/23 ━━━━━━━━━━━━━━━━━━━━

### 8. Evaluate Model
- Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. we will use the mean_squared_error function from Scikit-learn.

- Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

- Report the mean and the standard deviation of the mean squared errors.

In [100]:
# evaluate the model on the test data
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 249.7295  
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


250.1440887451172

 compute the mean squared error between the predicted concrete strength and the actual concrete strength.

In [90]:
from sklearn.metrics import mean_squared_error

In [101]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

250.1441189605334 0.0


In [102]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors without normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 150.2924041748047
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
MSE 2: 157.58871459960938
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MSE 3: 114.08971405029297
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
MSE 4: 96.42684936523438
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MSE 5: 77.27481842041016
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
MSE 6: 67.27429962158203
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MSE 7: 63.072998046875
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MSE 8: 46.21975326538086
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MSE 9: 49.36833953857422
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MSE 10: 47.42855453491211
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MSE 11: 45.64917755126953
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MSE 12: 43.30918502807617
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MSE 13: 50.76321792602539
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MSE 14: 48.91830062866211
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MSE 15: 44.63201141357422
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MSE 16: 39.81634140